In [1]:
# Import modules
import os
import sys
import numpy as np

import pandas as pd
import torch

import seqdata as sd
import seqpro as sp
import xarray as xr
import seqmodels as sm
from eugene.models import zoo

from eugene import models
from eugene import train

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
# Set wd
os.chdir("/cellar/users/aklie/data/datasets/deAlmeida_DrosophilaS2_UMI-STARR-seq")

In [3]:
# Set fold to eval on
date = "2023_12_19"
dataset = "basepair_res"
model = "DeepSTARR"
target_var = "hk"
train_var = "train_val"

# Set log dir
log_dir = f"models/{date}/{model}/{target_var}/{train_var}"
print(f"Log dir: {log_dir}")

# Set task
if target_var == "both":
    target_var = ["hk", "dev"]
elif target_var == "dev":
    target_var = ["dev"]
elif target_var == "hk":
    target_var = ["hk"]
else:
    raise ValueError(f"Invalid target: {target_var}")

Log dir: models/2023_12_19/DeepSTARR/hk/train_val


# Load SeqData

In [4]:
# Load SeqData
sdata = sd.open_zarr("training/2023_12_19/basepair_res/deepstarr_training_bpres.zarr")
sdata

<xarray.Dataset>
Dimensions:      (_sequence: 442866, _length: 2370, cov_sample: 2, cov_split: 1)
Coordinates:
  * cov_sample   (cov_sample) <U3 'dev' 'hk'
  * cov_split    (cov_split) <U2 'hk'
Dimensions without coordinates: _sequence, _length
Data variables:
    chrom        (_sequence) <U1 dask.array<chunksize=(55359,), meta=np.ndarray>
    chromEnd     (_sequence) int64 dask.array<chunksize=(55359,), meta=np.ndarray>
    chromStart   (_sequence) int64 dask.array<chunksize=(55359,), meta=np.ndarray>
    cleaned_seq  (_sequence, _length) |S1 dask.array<chunksize=(13840, 149), meta=np.ndarray>
    cov          (_sequence, cov_sample, _length) uint16 dask.array<chunksize=(2048, 1, 2370), meta=np.ndarray>
    dev          (_sequence, cov_split, _length) uint16 dask.array<chunksize=(2048, 1, 2370), meta=np.ndarray>
    hk           (_sequence, cov_split, _length) uint16 dask.array<chunksize=(2048, 1, 2370), meta=np.ndarray>
    seq          (_sequence, _length) |S1 dask.array<chunksize=(2048, 2370), meta=np.ndarray>
    strand       (_sequence) <U1 dask.array<chunksize=(55359,), meta=np.ndarray>
    train_val    (_sequence) bool dask.array<chunksize=(221433,), meta=np.ndarray>
Attributes:
    length_dim:    _length
    max_jitter:    128
    sequence_dim:  _sequence

In [5]:
# Load the training data into memory for faster training
sdata[['cleaned_seq', 'hk']].load()

<xarray.Dataset>
Dimensions:      (_sequence: 442866, _length: 2370, cov_split: 1)
Coordinates:
  * cov_split    (cov_split) <U2 'hk'
Dimensions without coordinates: _sequence, _length
Data variables:
    cleaned_seq  (_sequence, _length) |S1 b'C' b'T' b'C' b'T' ... b'A' b'G' b'C'
    hk           (_sequence, cov_split, _length) uint16 0 0 0 0 0 ... 0 0 0 0 0
Attributes:
    length_dim:    _length
    max_jitter:    128
    sequence_dim:  _sequence

In [6]:
sdata_train = sdata.sel(_sequence=(sdata["train_val"]==True).compute())
sdata_valid = sdata.sel(_sequence=(sdata["train_val"]==False).compute())

In [7]:
# Define training transformations
from eugene.dataload._augment import RandomRC

def seq_trans(x):
    x = x[..., 128:-128]
    np.char.upper(x)
    x = sp.ohe(x, sp.alphabets.DNA)
    x = x.swapaxes(1, 2)
    return torch.as_tensor(x.astype('f4'))

def cov_trans(x):
    x = x[..., 128+557:-128-557]
    return torch.as_tensor(x.astype('f4'))

# Get the train dataloader
dl = sd.get_torch_dataloader(
    sdata_train,
    sample_dims=['_sequence'],
    variables=['cleaned_seq', 'hk'],
    prefetch_factor=None,
    batch_size=32,
    transforms={
        'cleaned_seq': seq_trans,
        'hk': cov_trans,
    },
    return_tuples=True,
    shuffle=True,
)

In [8]:
# Test a batch
batch = next(iter(dl))
[x.shape for x in batch]

[torch.Size([32, 4, 2114]), torch.Size([32, 1, 1000])]

In [9]:
# Get the validation data
def seq_trans(x):
    x = x[..., 128:-128]
    np.char.upper(x)
    x = sp.ohe(x, sp.alphabets.DNA)
    x = x.swapaxes(1, 2)
    return torch.as_tensor(x.astype('f4'))

def cov_trans(x):
    x = x[..., 128+557:-128-557]
    return torch.as_tensor(x.astype('f4'))

X_valid = seq_trans(sdata_valid["cleaned_seq"].values)
y_valid = cov_trans(sdata_valid["hk"].values)
X_valid.shape, y_valid.shape

(torch.Size([40570, 4, 2114]), torch.Size([40570, 1, 1000]))

# Instantiate the model

In [10]:
from bpnetlite import BPNet

In [11]:
# Create the model, we need 1 channel for the signal tracks, and to trim to 1000 bp of input
model = BPNet(n_outputs=1, n_control_tracks=0, trimming=(2114 - 1000) // 2, name="bpnet.seqdata")
model


BPNet(
  (iconv): Conv1d(4, 64, kernel_size=(21,), stride=(1,), padding=(10,))
  (irelu): ReLU()
  (rconvs): ModuleList(
    (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
    (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
    (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
    (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(32,), dilation=(32,))
    (5): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(64,), dilation=(64,))
    (6): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(128,), dilation=(128,))
    (7): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(256,), dilation=(256,))
  )
  (rrelus): ModuleList(
    (0-7): 8 x ReLU()
  )
  (fconv): Conv1d(64, 1, kernel_size=(75,), stride=(1,), padding=(37,))
  (linear): Linear(in_features=64, out_features=1, bias=True)
)

In [12]:
# Test out a prediction batch
batch = next(iter(dl))
pred = model.forward(batch[0])
pred[0].shape, pred[1].shape

(torch.Size([32, 1, 1000]), torch.Size([32, 1]))

In [13]:
# Send the model to the GPU
model.cuda()

BPNet(
  (iconv): Conv1d(4, 64, kernel_size=(21,), stride=(1,), padding=(10,))
  (irelu): ReLU()
  (rconvs): ModuleList(
    (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
    (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
    (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
    (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(32,), dilation=(32,))
    (5): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(64,), dilation=(64,))
    (6): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(128,), dilation=(128,))
    (7): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(256,), dilation=(256,))
  )
  (rrelus): ModuleList(
    (0-7): 8 x ReLU()
  )
  (fconv): Conv1d(64, 1, kernel_size=(75,), stride=(1,), padding=(37,))
  (linear): Linear(in_features=64, out_features=1, bias=True)
)

In [14]:
# Quickly define your optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model

In [15]:
# Use the models fit_generator method to train the model
res = model.fit(
    dl,
    optimizer,
    X_valid=X_valid,
	X_ctl_valid=None,
    y_valid=y_valid,
    max_epochs=50
)

Epoch	Iteration	Training Time	Validation Time	Training MNLL	Training Count MSE	Validation MNLL	Validation Profile Pearson	Validation Count Pearson	Validation Count MSE	Saved?


0	0	1.5878	7.7605	0.0	0.1349	0.0	0.0	0.0	1.0647	True
0	100	13.1921	6.7825	0.0	0.0	0.0	0.0	0.0	0.0	True
0	200	12.0337	6.7967	0.0	0.0	0.0	0.0	0.0	0.0	True
0	300	12.1026	6.8003	0.0	0.0	0.0	0.0	0.0	0.0	True
0	400	11.9798	6.7974	0.0	0.0	0.0	0.0	0.0	0.0	True
0	500	12.4903	6.8063	0.0	0.0	0.0	0.0	0.0	0.0	True
0	600	12.4673	6.8271	0.0	0.0	0.0	0.0	0.0	0.0	True
0	700	12.3654	6.8103	0.0	0.0	0.0	0.0	0.0	0.0	True
0	800	12.0435	6.8225	0.0	0.0	0.0	0.0	0.0	0.0	True
0	900	12.0361	6.8287	0.0	0.0	0.0	0.0	0.0	0.0	True
0	1000	12.4862	6.8337	0.0	0.0	0.0	0.0	0.0	0.0	True
0	1100	12.2984	6.8318	0.0	0.0	0.0	0.0	0.0	0.0	True
0	1200	12.6002	6.8287	0.0	0.0	0.0	0.0	0.0	0.0	False
0	1300	11.9139	6.8187	0.0	0.0	0.0	0.0	0.0	0.0	True
0	1400	11.8375	6.8302	0.0	0.0	0.0	0.0	0.0	0.0	True
0	1500	11.7227	6.8087	0.0	0.0	0.0	0.0	0.0	0.0	True
0	1600	12.5324	6.8407	0.0	0.0	0.0	0.0	0.0	0.0	True
0	1700	11.8153	6.8246	0.0	0.0	0.0	0.0	0.0	0.0	False
0	1800	11.9644	6.8225	0.0	0.0	0.0	0.0	0.0	0.0	True
0	1900	11.8081	6.8266	0.0	0.0	0.0	0.

# DONE!

---